In [ ]:
import torch
from torch.utils.data import random_split, DataLoader
from sklearn.metrics import roc_auc_score, roc_curve
from data_loader import SimpleDatasetLoader
import matplotlib.pyplot as plt
from torch import nn, optim
from models.FC import *
import pandas as pd

In [ ]:
df = pd.read_csv("../data/scoring_case.csv")

for column in df.columns:
    if str(df[column].dtype) == 'object':
        df[column] = pd.factorize(df[column])[0]

In [ ]:
df.dropna(axis=1, thresh=int(0.8 * df.shape[0]), inplace=True)
df = df.drop(["FLAG_MOBIL", "SK_ID_CURR"], axis=1)
1 - df.isna().sum() / len(df)

In [ ]:
for column in df.columns:
    if column == 'TARGET':
        continue

    df[column] = df[column].fillna(round(df[column].mode()))

In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)

In [ ]:
dataset = SimpleDatasetLoader(df)
train_amount_data = len(dataset) * 2 // 3
train, test = random_split(dataset, [train_amount_data, len(dataset) - train_amount_data])

In [ ]:
train_loader = DataLoader(train, batch_size=32)
test_loader = DataLoader(test, batch_size=32)

In [ ]:
model = FC()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
num_epochs = 15

In [ ]:
train_model(model, train_loader, criterion, optimizer, num_epochs)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
title = ["train", "test"]
for i, loader in enumerate([train_loader, test_loader]):
    prediction, target_list = get_prediction(model, loader)
    logit_roc_aut = roc_auc_score(target_list, prediction)
    fpr, tpr, thresholds = roc_curve(target_list, prediction)
    
    axs[i].plot(fpr, tpr, logit_roc_aut)
    axs[i].plot([0, 1], [0, 1], '--')
    axs[i].set_title(f"{title[i]} - ({round(logit_roc_aut, 4)})")
plt.show()

In [ ]:
model.load_state_dict(torch.load("../weigh/2.pt"))

In [ ]:
# torch.save(model.state_dict(), "../weigh/2.pt")